In [1]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [3]:
COLLECTION_NAME = 'data_true_4'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[89005882,
 517707191,
 1313482693208924167,
 637211551,
 1390735684869181440,
 66886804,
 68270085,
 156147230,
 177273594,
 39859832,
 531565758,
 72994171,
 1332453254958100490,
 51183779,
 44202038,
 811454879101231104,
 2238402254,
 1380667196389662721,
 838081120101150721,
 309113313,
 19650266,
 19650266,
 335206333,
 21527796,
 465827161,
 1359888759286870024,
 344778430,
 1978165356,
 43730427,
 882445459,
 336702784,
 263016089,
 471470510,
 265125249,
 527735717,
 1377460890522460162,
 1361479387745558534,
 1327898728527765505,
 2700578686,
 1311550658978275328,
 467335200,
 1388231987492687877,
 16162664,
 1359903282022518787,
 42102154,
 327659422,
 26443360,
 1339584626671513608,
 79318262,
 189719210,
 1289797717,
 474464413,
 1046906136866279424,
 882445459,
 1133833482122145792,
 457984599,
 55821927,
 602274330,
 3246358172,
 1335012527466557446,
 3146781148,
 1204749918692413440,
 757985417975050240,
 3359231747,
 3245441164,
 1347906422206582788,
 1074467761,
 10744

In [4]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{25754433, 84111298, 457984599, 61558438, 519897737, 2300835530, 32913997, 882445459, 44202038, 474464413, 89005882, 335206333, 344778430}


NameError: name 'db' is not defined

In [5]:
users= [25754433, 84111298, 457984599, 61558438, 519897737, 2300835530, 32913997, 882445459, 44202038, 474464413, 89005882, 335206333, 344778430]

In [6]:
for user in users:
    print(user, get_followers_len(user, db))

25754433 0
84111298 0
457984599 0
61558438 0
519897737 0
2300835530 0
32913997 0
882445459 0
44202038 0
474464413 0
89005882 0
335206333 0
344778430 0


In [7]:
for user in users:
    print(user, get_cascade_size(user, COLLECTION_NAME, db))

25754433 1
84111298 1
457984599 659
61558438 2
519897737 8
2300835530 1
32913997 4
882445459 6
44202038 3
474464413 1
89005882 11
335206333 2
344778430 2


In [8]:
# Check if the text is same
db[COLLECTION_NAME].distinct('retweet_count')

[0, 1, 2, 3, 7, 8, 10, 12, 167, 273, 363]

In [9]:
db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')

[1, 2, 3, 7, 8, 10, 12, 167, 273, 363]

In [10]:
db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')

[9, 1405]

In [11]:
# Check if the text is same
cache = db[COLLECTION_NAME].distinct('retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweet_count':x})['text'])
    print('*'*5)

[10, 12, 167, 273, 363]
5 days ago the Biden Admin told Americans worried about gas shortages:  "If you drive an electric car this would no… https://t.co/jCKLJ9xYEA
*****
"If you drive an electric car, this would not be affecting you, clearly."

This is the same woman who has millions… https://t.co/ZqAsx2unqC
*****
Nolte: What's Biden's response to the gas crisis? "If you drive an electric car, this would not be affecting you, c… https://t.co/4Bg4JbXnsZ
*****
Nolte: What's Biden's response to the gas crisis? "If you drive an electric car, this would not be affecting you, c… https://t.co/717lOsHBOL
*****
Nolte: What's Biden's response to the gas crisis? "If you drive an electric car, this would not be affecting you, c… https://t.co/faLW5NuGrN
*****


In [12]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.retweet_count':x})['id'])
    print('*'*5)

[10, 12, 167, 273, 363]
1394701420474118150
*****
1392196597598068739
*****
1393217509202042884
*****
1393245182670290945
*****
1393362887767384065
*****


In [13]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.quoted_status.retweet_count':x})['retweeted_status.quoted_status.id'])
    print('*'*5)

[9, 1405]


KeyError: 'retweeted_status.quoted_status.id'